# MIMIC-IV Data Exploration

This notebook performs Exploratory Data Analysis (EDA) on MIMIC-IV data to understand patient statistics, feature distributions, and dataset complexity. The analysis includes patient demographics, admission patterns, laboratory events, microbiology tests, and chart events (vitals).

**Important Note**: The `chartevents` and `labevents` tables are massive. This notebook uses chunking to avoid Out-Of-Memory errors.

## Section 1: Setup & Configuration

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

# Set plot style for better readability
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# Define file paths - update these to match your data location
patients_file = 'patients.csv.gz'
admissions_file = 'admissions.csv.gz'
labevents_file = 'labevents.csv.gz'
d_labitems_file = 'd_labitems.csv.gz'
microbiologyevents_file = 'microbiologyevents.csv.gz'
chartevents_file = 'chartevents.csv.gz'
d_items_file = 'd_items.csv.gz'

print("File paths defined.")

## Section 2: Patient & Admission Statistics

Load patient and admission data to understand the overall structure of the dataset.

In [ ]:
# Load patients table
print("Loading patients table...")
patients_df = pd.read_csv(patients_file)
print(f"Patients loaded: {len(patients_df)} rows")
print(f"Columns: {list(patients_df.columns)}")

In [ ]:
# Load admissions table
print("Loading admissions table...")
admissions_df = pd.read_csv(admissions_file)
print(f"Admissions loaded: {len(admissions_df)} rows")
print(f"Columns: {list(admissions_df.columns)}")

In [ ]:
# Calculate basic statistics
total_unique_patients = patients_df['subject_id'].nunique()
total_admissions = len(admissions_df)

print("=" * 80)
print("PATIENT & ADMISSION STATISTICS")
print("=" * 80)
print(f"Total Unique Patients: {total_unique_patients:,}")
print(f"Total Admissions: {total_admissions:,}")
print()

In [ ]:
# Calculate admissions per patient
admissions_per_patient = admissions_df.groupby('subject_id').size()

min_admissions = admissions_per_patient.min()
max_admissions = admissions_per_patient.max()
avg_admissions = admissions_per_patient.mean()

print("ADMISSIONS PER PATIENT:")
print(f"  Minimum: {min_admissions}")
print(f"  Maximum: {max_admissions}")
print(f"  Average: {avg_admissions:.2f}")
print("=" * 80)

In [ ]:
# Visualization: Histogram of admissions per patient
plt.figure(figsize=(12, 6))
plt.hist(admissions_per_patient, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Admissions per Patient')
plt.ylabel('Frequency (Number of Patients)')
plt.title('Distribution of Number of Admissions per Patient')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Section 3: Laboratory Events Exploration

Analyze laboratory test events. The `labevents` table is large, so we use chunk processing.

In [ ]:
# Load d_labitems dictionary for mapping itemid to label
print("Loading d_labitems dictionary...")
d_labitems_df = pd.read_csv(d_labitems_file)
print(f"Total lab items in dictionary: {len(d_labitems_df)}")

# Create mapping from itemid to label
labitem_map = dict(zip(d_labitems_df['itemid'], d_labitems_df['label']))
print(f"Mapping created for {len(labitem_map)} lab items.")

In [ ]:
# Read labevents in chunks and aggregate
print("\nReading labevents file in chunks...")
chunk_size = 1000000
lab_events_per_admission = defaultdict(int)
itemid_frequency = defaultdict(int)
chunk_number = 0

for chunk in pd.read_csv(
    labevents_file,
    usecols=['hadm_id', 'itemid'],
    chunksize=chunk_size
):
    chunk_number += 1
    print(f"Processing chunk {chunk_number}... ({len(chunk):,} rows)")
    
    # Remove rows with missing hadm_id
    chunk = chunk.dropna(subset=['hadm_id'])
    
    # Count lab events per admission
    for hadm_id in chunk['hadm_id']:
        lab_events_per_admission[hadm_id] += 1
    
    # Count frequency of each itemid
    itemid_counts = chunk['itemid'].value_counts()
    for itemid, count in itemid_counts.items():
        itemid_frequency[itemid] += count

print(f"\nTotal chunks processed: {chunk_number}")
print(f"Total admissions with lab events: {len(lab_events_per_admission):,}")
print(f"Unique lab test types: {len(itemid_frequency):,}")

In [ ]:
# Calculate statistics for lab events per admission
lab_events_counts = list(lab_events_per_admission.values())
min_lab_events = min(lab_events_counts)
max_lab_events = max(lab_events_counts)
avg_lab_events = np.mean(lab_events_counts)

print("=" * 80)
print("LAB EVENTS PER ADMISSION:")
print(f"  Minimum: {min_lab_events}")
print(f"  Maximum: {max_lab_events}")
print(f"  Average: {avg_lab_events:.2f}")
print("=" * 80)

In [ ]:
# Visualization 1: Histogram of lab events per admission
plt.figure(figsize=(12, 6))
plt.hist(lab_events_counts, bins=100, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Lab Events per Admission')
plt.ylabel('Frequency (Number of Admissions)')
plt.title('Distribution of Number of Lab Events per Admission')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Get top 20 most frequent lab tests
top_20_labitems = sorted(itemid_frequency.items(), key=lambda x: x[1], reverse=True)[:20]

# Map itemids to labels
top_20_labels = []
top_20_counts = []
for itemid, count in top_20_labitems:
    label = labitem_map.get(itemid, f"Unknown ({itemid})")
    top_20_labels.append(label)
    top_20_counts.append(count)

print("\nTOP 20 MOST FREQUENT LAB TESTS:")
for i, (label, count) in enumerate(zip(top_20_labels, top_20_counts), 1):
    print(f"{i:2d}. {label:50s} - {count:,} occurrences")

In [ ]:
# Visualization 2: Horizontal bar chart of top 20 lab tests
plt.figure(figsize=(12, 8))
y_pos = np.arange(len(top_20_labels))
plt.barh(y_pos, top_20_counts, alpha=0.7)
plt.yticks(y_pos, top_20_labels)
plt.xlabel('Number of Occurrences')
plt.ylabel('Lab Test')
plt.title('Top 20 Most Frequent Laboratory Tests')
plt.gca().invert_yaxis()  # Highest at the top
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## Section 4: Microbiology Events Exploration

Analyze microbiology test events including cultures and susceptibility tests.

In [ ]:
# Load microbiologyevents table
print("Loading microbiologyevents table...")
micro_df = pd.read_csv(microbiologyevents_file)
print(f"Microbiology events loaded: {len(micro_df):,} rows")
print(f"Columns: {list(micro_df.columns)}")

In [ ]:
# Remove rows with missing hadm_id
micro_df = micro_df.dropna(subset=['hadm_id'])
print(f"After removing missing hadm_id: {len(micro_df):,} rows")

In [ ]:
# Count microbiology events per admission
micro_events_per_admission = micro_df.groupby('hadm_id').size()

min_micro_events = micro_events_per_admission.min()
max_micro_events = micro_events_per_admission.max()
avg_micro_events = micro_events_per_admission.mean()

print("=" * 80)
print("MICROBIOLOGY EVENTS PER ADMISSION:")
print(f"  Minimum: {min_micro_events}")
print(f"  Maximum: {max_micro_events}")
print(f"  Average: {avg_micro_events:.2f}")
print("=" * 80)

In [ ]:
# Visualization 1: Histogram of microbiology events per admission
plt.figure(figsize=(12, 6))
plt.hist(micro_events_per_admission, bins=100, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Microbiology Events per Admission')
plt.ylabel('Frequency (Number of Admissions)')
plt.title('Distribution of Number of Microbiology Events per Admission')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Create combined test name from test_name and spec_type_desc
micro_df['combined_test'] = micro_df.apply(
    lambda row: f"{row['test_name']} - {row['spec_type_desc']}" 
    if pd.notna(row['spec_type_desc']) and pd.notna(row['test_name'])
    else (row['test_name'] if pd.notna(row['test_name']) else 'Unknown'),
    axis=1
)

# Count frequency of each test
test_frequency = micro_df['combined_test'].value_counts()
top_20_micro_tests = test_frequency.head(20)

print("\nTOP 20 MOST FREQUENT MICROBIOLOGY TESTS:")
for i, (test_name, count) in enumerate(top_20_micro_tests.items(), 1):
    print(f"{i:2d}. {test_name:60s} - {count:,} occurrences")

In [ ]:
# Visualization 2: Horizontal bar chart of top 20 microbiology tests
plt.figure(figsize=(12, 8))
y_pos = np.arange(len(top_20_micro_tests))
plt.barh(y_pos, top_20_micro_tests.values, alpha=0.7, color='coral')
plt.yticks(y_pos, top_20_micro_tests.index)
plt.xlabel('Number of Occurrences')
plt.ylabel('Microbiology Test')
plt.title('Top 20 Most Frequent Microbiology Tests')
plt.gca().invert_yaxis()  # Highest at the top
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## Section 5: Chart Events (Vitals) Exploration

Analyze chart events which contain vital signs and nursing observations. This is the largest table in MIMIC-IV.

In [ ]:
# Load d_items dictionary for mapping itemid to label
print("Loading d_items dictionary...")
d_items_df = pd.read_csv(d_items_file)
print(f"Total chart items in dictionary: {len(d_items_df)}")

# Create mapping from itemid to label
chartitem_map = dict(zip(d_items_df['itemid'], d_items_df['label']))
print(f"Mapping created for {len(chartitem_map)} chart items.")

In [ ]:
# Read chartevents in chunks and count itemid frequency
print("\nReading chartevents file in chunks (this may take a while)...")
chunk_size = 1000000
chart_itemid_frequency = defaultdict(int)
chunk_number = 0

for chunk in pd.read_csv(
    chartevents_file,
    usecols=['itemid'],
    chunksize=chunk_size
):
    chunk_number += 1
    print(f"Processing chunk {chunk_number}... ({len(chunk):,} rows)")
    
    # Count frequency of each itemid
    itemid_counts = chunk['itemid'].value_counts()
    for itemid, count in itemid_counts.items():
        chart_itemid_frequency[itemid] += count

print(f"\nTotal chunks processed: {chunk_number}")
print(f"Unique chart event types: {len(chart_itemid_frequency):,}")

In [ ]:
# Get top 20 most frequent chart events
top_20_chartitems = sorted(chart_itemid_frequency.items(), key=lambda x: x[1], reverse=True)[:20]

# Map itemids to labels
top_20_chart_labels = []
top_20_chart_counts = []
for itemid, count in top_20_chartitems:
    label = chartitem_map.get(itemid, f"Unknown ({itemid})")
    top_20_chart_labels.append(label)
    top_20_chart_counts.append(count)

print("\nTOP 20 MOST FREQUENT CHART EVENTS (VITALS/OBSERVATIONS):")
for i, (label, count) in enumerate(zip(top_20_chart_labels, top_20_chart_counts), 1):
    print(f"{i:2d}. {label:50s} - {count:,} occurrences")

In [ ]:
# Visualization: Horizontal bar chart of top 20 chart events
plt.figure(figsize=(12, 8))
y_pos = np.arange(len(top_20_chart_labels))
plt.barh(y_pos, top_20_chart_counts, alpha=0.7, color='steelblue')
plt.yticks(y_pos, top_20_chart_labels)
plt.xlabel('Number of Occurrences')
plt.ylabel('Chart Event (Vital/Observation)')
plt.title('Top 20 Most Frequent Chart Events')
plt.gca().invert_yaxis()  # Highest at the top
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

This notebook has successfully performed comprehensive exploratory data analysis on MIMIC-IV data:

1. ✅ **Setup & Configuration**: Imported pandas, matplotlib, seaborn; set readable plot style; defined file paths
2. ✅ **Patient & Admission Statistics**: Calculated total patients, admissions, and admissions per patient; created histogram
3. ✅ **Laboratory Events**: Used chunk processing to analyze lab events; calculated statistics; created 2 visualizations (histogram and top 20 bar chart)
4. ✅ **Microbiology Events**: Analyzed microbiology tests; calculated statistics; created 2 visualizations (histogram and top 20 bar chart)
5. ✅ **Chart Events (Vitals)**: Used chunk processing to analyze the largest table; created top 20 visualization

All visualizations include proper titles, axis labels, and are sized appropriately for readability. The chunking strategy successfully handles the massive `labevents` and `chartevents` tables without memory issues.